In [2]:
import sys
print(sys.executable)


c:\own projects\ArchiveLLM\venv\Scripts\python.exe


In [3]:
import os
import openai
import json
from dotenv import load_dotenv

load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")
#setup & API laden

In [4]:
import pandas as pd

# Lies deine CSV oder kombiniere alle JSONs (je nach Aufbau)
df = pd.read_csv("../../output/alle_fragen_gesamt.csv")


# Häufigstes Thema ermitteln (Beispiel)
top_themen = df["thema"].value_counts().head(3).index.tolist()

# Pro Thema: Aufgaben sammeln
themen_aufgaben = {
    thema: df[df["thema"] == thema]["frage"].tolist()
    for thema in top_themen
}
#Häufige Themen und zugehörige Originalaufgaben sammeln

In [ ]:
def generate_prompt(thema, fragen_liste):
    #eindeutige Fragen, maximal 5, schön formatiert 
    joined = "\n".join(set(fragen_liste[:5]))
    return f"""Hier sind fünf echte Prüfungsfragen zum Thema "{thema}":
{joined}

Erstelle daraus drei neue, inhaltlich vergleichbare Übungsfragen im selben Stil."""


from openai import OpenAI

client = OpenAI()

def call_gpt(prompt):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.7
    )
    return response.choices[0].message.content

In [6]:
results = {}

for thema, fragen in themen_aufgaben.items():
    prompt = generate_prompt(thema, fragen)
    output = call_gpt(prompt)
    results[thema] = {
        "prompt": prompt,
        "generated_tasks": output
    }

# speichern
os.makedirs("generated_tasks", exist_ok=True)
with open("generated_tasks/aufgaben_llm.json", "w", encoding="utf-8") as f:
    json.dump(results, f, indent=2, ensure_ascii=False)
#Loop über alle Themen

In [8]:
#  Exportiere die generierten Aufgaben als PDF
from fpdf import FPDF
import json

# Lade die JSON-Datei
with open("generated_tasks/aufgaben_llm.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# PDF vorbereiten
pdf = FPDF()
pdf.set_auto_page_break(auto=True, margin=15)
pdf.add_page()
pdf.set_font("Arial", size=12)

pdf.set_title("Generierte Prüfungsaufgaben")

for thema, inhalt in data.items():
    pdf.set_font("Arial", 'B', size=14)
    pdf.cell(0, 10, f"Thema: {thema}", ln=True)
    
    pdf.set_font("Arial", '', size=12)
    pdf.multi_cell(0, 10, f"Prompt:\n{inhalt['prompt']}\n", align="L")
    pdf.multi_cell(0, 10, f"Generierte Aufgaben:\n{inhalt['generated_tasks']}\n\n", align="L")

# PDF speichern
output_path = "generated_tasks/aufgaben_llm_export.pdf"
pdf.output(output_path)

print(f"✅ Export abgeschlossen: {output_path}")


✅ Export abgeschlossen: generated_tasks/aufgaben_llm_export.pdf
